In [3]:
from sshtunnel import SSHTunnelForwarder
import pymysql
import time
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline


import numpy as np
import matplotlib.patches as mpatches

from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error 
from sklearn.metrics import r2_score #R square
from sklearn.neural_network import MLPRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.model_selection import TimeSeriesSplit

from sklearn.svm import SVC
import xgboost as xgb
import lightgbm as lgbm
from sklearn.svm import SVR
from sklearn.neural_network import MLPClassifier
from sklearn import metrics
from sklearn.model_selection import cross_val_score
import pca as pca

# ignore warnings
import warnings
warnings.filterwarnings('ignore')
from matplotlib.axes._axes import _log as matplotlib_axes_logger
matplotlib_axes_logger.setLevel('ERROR')

## code used to clean and create useful columns for each line

In [ ]:
# df.read_csv('baseline/line270_BASELINE.csv')

In [ ]:
# #   add extra features: 'date', 'month', 'weekday', 'dep_hour', 'arr_hour'
# df.DAYOFSERVICE = pd.to_datetime(df.DAYOFSERVICE) 
# df['date'] = df['DAYOFSERVICE'].dt.day
# df['month'] = df['DAYOFSERVICE'].dt.month
# df['weekday'] = df['DAYOFSERVICE'].dt.weekday
# df['dep_hour'] = (df['ACTUALTIME_DEP'] // 3600) - 1
# df['arr_hour'] = (df['ACTUALTIME_ARR'] // 3600) - 1
    
# #   drop the rows which arr_hour < 25
# df = df.loc[df.arr_hour <= 24]


# # Sort and reindex the df so that each trip is in order
# df = df.sort_values(['DIRECTION', 'DAYOFSERVICE','TRIPID', 'PROGRNUMBER'], ascending=[True, True, True, True])
# df = df.reset_index(drop=True)


# df.loc[(df.PROGRNUMBER > df.PROGRNUMBER.shift(1)) &
# (df.TRIPID == df.TRIPID.shift(1)) & 
# (df.DIRECTION == df.DIRECTION.shift(1)) &
# (df.DAYOFSERVICE == df.DAYOFSERVICE.shift(1))
# , 'journey_time'] = (df.ACTUALTIME_ARR - df.ACTUALTIME_ARR.shift(1))

    
# df.loc[(df.PROGRNUMBER > df.PROGRNUMBER.shift(1)) &
#           (df.TRIPID == df.TRIPID.shift(1)) & 
#           (df.DIRECTION == df.DIRECTION.shift(1)) &
#           (df.DAYOFSERVICE == df.DAYOFSERVICE.shift(1))
#           , 'planned_journey_time'] = (df.PLANNEDTIME_ARR - df.PLANNEDTIME_ARR.shift(1))
    

    
#     # Calc dwell time
#     # df_rev1_1['dwell_time'] = df_rev1_1.ACTUALTIME_DEP - df_rev1_1.ACTUALTIME_ARR

#     # Get the previous stop's id
# df.loc[(df.PROGRNUMBER > df.PROGRNUMBER.shift(1)) &
#               (df.TRIPID == df.TRIPID.shift(1)) & 
#               (df.DIRECTION == df.DIRECTION.shift(1)) &
#               (df.DAYOFSERVICE == df.DAYOFSERVICE.shift(1))
#               , 'prev_stop_id'] = df.STOPPOINTID.shift(1)

#     # Create a segment id by concatenating prev stop id and currnet stop id
# df["segment_id"] = df['prev_stop_id'].map(str) + "-" + df['STOPPOINTID'].map(str)

# # If the progrnumber is 1, that is the first stop, ie there is no previous stop
# # set the journey time, seg id and prev stop to NaN
# df.loc[df['PROGRNUMBER'] == 1, "journey_time"] = np.nan
# df.loc[df['PROGRNUMBER'] == 1, "prev_stop_id"] = np.nan
# df.loc[df['PROGRNUMBER'] == 1, "segment_id"] = np.nan
    
    
#     #  add isPeaktime feature
#     #  value 1: 6:30 - 9:30, 15:30 - 18:30 
#     #  value: otherwise
# df['isPeaktime'] = 0
# df['isPeaktime'] = np.where(((df.ACTUALTIME_DEP >= 23400) & (df.ACTUALTIME_DEP <= 34200)) |
#                                ((df.ACTUALTIME_DEP >= 55800) & (df.ACTUALTIME_DEP <= 66600))
#                                , 1, df['isPeaktime'])

In [ ]:
# df['difference']=df['planned_journey_time'] - df['journey_time']

In [ ]:
# df = df.drop(columns=['Unnamed: 0','STOPPOINTID','PROGRNUMBER',
#                       'TRIPID','ACTUALTIME_DEP','ACTUALTIME_ARR',
#                       'prev_stop_id','dep_hour','PLANNEDTIME_ARR','PLANNEDTIME_DEP',
#                       'DIRECTION'       
#                      ])

In [ ]:
# def delete_null_row(df):
#     lis = ['journey_time', 'planned_journey_time','segment_id']
#     for i in lis:
#         df.drop(df[df[i].isnull().values==True].index,inplace = True)
# delete_null_row(df)

In [ ]:
# df["mean_difference"]=df["difference"].mean()

In [ ]:
# df.to_csv('baseline/line270_BASELINE.csv')

## code used to get the mean and absolute mean between journey time difference(df['planned_journey_time'] - df['journey_time']) for all routes

In [4]:
import glob

path = r'baseline' # use your path
all_files = glob.glob(path + "/*.csv")

li = []

for filename in all_files:
    df = pd.read_csv(filename, index_col=None, header=0)
    li.append(df)

frame = pd.concat(li, axis=0, ignore_index=True)

In [5]:
frame

,Unnamed: 0,DAYOFSERVICE,LINEID,date,month,weekday,arr_hour,journey_time,planned_journey_time,segment_id,isPeaktime,difference,mean_difference
0,1,2018-01-01,65,1,1,0,8,130.0,146.0,7564.0-4521,0,16.0,-7.266759
1,2,2018-01-01,65,1,1,0,8,90.0,103.0,4521.0-1283,0,13.0,-7.266759
2,3,2018-01-01,65,1,1,0,8,39.0,37.0,1283.0-4456,0,-2.0,-7.266759
3,4,2018-01-01,65,1,1,0,8,22.0,34.0,4456.0-1284,0,12.0,-7.266759
4,5,2018-01-01,65,1,1,0,8,61.0,96.0,1284.0-1285,0,35.0,-7.266759
...,...,...,...,...,...,...,...,...,...,...,...,...,...
108187504,141452,2018-12-21,33X,21,12,4,6,273.0,212.0,7398.0-6252,1,-61.0,-12.802967
108187505,141453,2018-12-21,33X,21,12,4,6,161.0,275.0,6252.0-302,1,114.0,-12.802967
108187506,141454,2018-12-21,33X,21,12,4,6,85.0,119.0,302.0-4495,1,34.0,-12.802967
108187507,141455,2018-12-21,33X,21,12,4,7,253.0,230.0,4495.0-2809,1,-23.0,-12.802967


In [6]:
frame["difference"].mean()

-6.309524004291475

In [7]:
frame["difference"].abs().mean()

26.07248421811801